<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis Integration with Customer360
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Complaints Analysis Integration with Customer360 is a comprehensive approach to managing customer complaints and feedback within the framework of a Customer 360-degree view using <b>Teradata Vantage</b> and <b>Azure OpenAI</b>. This integration aims to provide a seamless and personalized customer experience by leveraging data from various sources, including CRM systems, marketing platforms, and social media.</p> <p style='font-size:16px;font-family:Arial;color:#00233C'>The key components of this integration include:</p> 

<ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Customer 360 Data Manager</b>: Responsible for managing and maintaining a comprehensive view of customer data, including collection, integration, and analysis from multiple sources.</li> <li><b>Complaints Management Dashboard</b>: Analyzes customer complaints, providing insights into complaint volume, trends, and resolution progress.</li> <li><b>Customer Insights</b>: Tools for gaining insights into customer behavior and preferences, enabling targeted marketing campaigns and informed business decisions.</li> </ol> <p style='font-size:16px;font-family:Arial;color:#00233C'>The benefits of this integration include:</p> <ol style='font-size:16px;font-family:Arial;color:#00233C'> <li><b>Improved Customer Experience</b>: By integrating complaints analysis with Customer 360, businesses can address customer complaints more effectively, leading to increased customer satisfaction and loyalty.</li> <li><b>Data-Driven Decision Making</b>: The integration provides a centralized platform for analyzing customer data, enabling businesses to make informed decisions about product development, marketing strategies, and customer engagement.</li> <li><b>Enhanced Customer Insights</b>: The comprehensive view of customer data allows businesses to better understand customer needs and preferences, leading to more targeted and effective marketing efforts.</li> </ol> 


<p style='font-size:16px;font-family:Arial;color:#00233C'>By integrating complaints analysis with Customer 360, businesses can create a more comprehensive and personalized customer experience, driving business growth and customer satisfaction.</p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environement</li>
    <li>Connect to Vantage</li>
    <li>Configuring Azure OpenAI</li>
    <li>Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summerization</li>
    <li>Integrated data with customer 360</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization
import plotly.express as px

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaint_Summarization.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;background-color:#00233C;">
<b style='font-size:20px;font-family:Arial;color:#00233C'>3. Configuring Azure OpenAI</b>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Before proceeding, you need to provide the following information:</p>
<ul style='font-size:16px;font-family:Arial;color:#00233C'>
<li><b>Endpoint</b>: Enter your Azure OpenAI deployment endpoint.</li>
<li><b>Azure OpenAI API Key</b>: Enter your Azure OpenAI API Key.</li>
</ul>
<p style='font-size:16px;font-family:Arial;color:#00233C'>If you haven't retrieved your API Key and Endpoint yet, follow the instructions <a href="https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python" target="_blank" style="color:#0066CC;text-decoration:none;"><b>here</b></a>.</p>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Don't have an Azure OpenAI resource yet? Follow this guide:</p>
<a href="./Azure-OpenAI.ipynb" style="text-decoration:none;" target="_blank">
    <button style="font-size:16px;font-family:Arial;color:#fff;background-color:#00233C;border:none;border-radius:5px;cursor:pointer;height:50px;line-height:50px;display:flex;align-items:center;">
        Azure OpenAI Guide <span style="margin-left:10px;">&#8658;</span>
    </button>
</a>

In [ ]:
# Prompt user for Azure OpenAI endpoint securely
os.environ["ENDPOINT"] = getpass.getpass(prompt="\nPlease enter your Azure OpenAI endpoint(gpt-4o-mini): ")
# Prompt user for Azure OpenAI API key securely
os.environ["API_KEY"] = getpass.getpass(prompt="\nPlease enter your Azure OpenAI API key(gpt-4o-mini): ")

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>3.1 Initialize the Azure OpenAI API request</b>

In [ ]:
headers = {
    "Content-Type": "application/json",
    "api-key": os.environ["API_KEY"],
}

def get_payload(prompt):
    # Payload for the request
    payload = {
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": "You are an assistant that analyzes sentiment of a text and gives reasoning for the categorization as well.\n"
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            }
          ]
        }
      ],
      "temperature": 0.7,
      "top_p": 1,
      "max_tokens": 512
    }

    return(payload)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summarization</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis, Topic Modelling and Complaint Summarization using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>Sentiment Analysis Using Large Language Models (LLMs) is a cutting-edge approach to understanding customer opinions and emotions expressed through text-based data. This advanced technique leverages the capabilities of LLMs to accurately identify and categorize sentiment as positive, negative, or neutral, providing businesses with valuable insights into customer perceptions and preferences.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Reasoning with a Chain of Thought</b>: Imagine you're trying to solve a problem. With a large language model, you start with an initial idea or question. Then, you use the model's capabilities to explore related concepts, gradually connecting them together. Each step builds upon the previous one, leading you closer to understanding or solving the problem. It's like putting together puzzle pieces, one by one, until you see the whole picture.</p>

In [ ]:
customer_data = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Customer_360_Details'))
customer_data

In [ ]:
complaints_data = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Customer_360_Complaints'))
complaints_data

In [ ]:
pd_df = complaints_data.to_pandas()
pd_df['Sentiment'] = ""
pd_df['Topic'] = ""
pd_df['Summary'] = ""

In [ ]:
# Sentiment
for i in tqdm(range(len(pd_df))):
    prompt = f'''
        User prompt:
        The following is text from a review:

        “{pd_df['consumer_complaint_narrative'][i]}”

        Categorize the review as one of the following:

        Positive
        Negative
        Neutral
    '''

    # Send request
    try:
        response = requests.post(os.environ["ENDPOINT"], headers = headers, json = get_payload(prompt))
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    output = response.json()['choices'][0]['message']['content']
    pd_df['Sentiment'][i] = output.strip()

In [ ]:
# Topic
for i in tqdm(range(len(pd_df))):
    prompt = f'''
        User prompt:
        The following is text from a complaint:

        “{pd_df['consumer_complaint_narrative'][i]}”

        Identify the topic of the complaint and categorize into one of the following topics. Only output one of the following options:

        - Mortgage Application
        - Payment Trouble
        - Mortgage Closing
        - Report Inaccuracy
        - Payment Struggle

        Return just one of the above options
'''
    # Send request
    try:
        response = requests.post(os.environ["ENDPOINT"], headers = headers, json = get_payload(prompt))
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    output = response.json()['choices'][0]['message']['content']
    pd_df['Topic'][i] = output.strip()

In [ ]:
# Summary
for i in tqdm(range(len(pd_df))):
    prompt = f'''
        The following is text from a Bank Review:
        “{pd_df['consumer_complaint_narrative'][i]}”
        Summarize the Bank Review in one sentence
    '''

    # Send request
    try:
        response = requests.post(os.environ["ENDPOINT"], headers = headers, json = get_payload(prompt))
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    output = response.json()['choices'][0]['message']['content']
    pd_df['Summary'][i] = output.strip()

In [ ]:
# Convert Teradata DataFrame to Pandas DataFrame
customer_df = customer_data.to_pandas().reset_index()

# Rename the columns to match for joining
customer_df = customer_df.rename(columns = {"Customer Identifier": "Customer_ID"})

# Inner join:
combined_df = customer_df.merge(pd_df, how = "inner", on="Customer_ID")

In [ ]:
combined_df["Bank Strategy"] = ['Wealth Manager to contact customer immediately',
                                'Send Policy Letter from Mortgage Servicing',
                                'Send Policy Letter from Executive Office',
                                'Mortgage Banker to follow-up with Title Company for documentation and contact customer',
                                'Branch Manager to contact customer immediately']

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Integrated data with customer 360</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following is an example of the output from LLM integrated with existing customer360 data. Please scroll to the right to see all the columns.</p>

In [ ]:
pd.set_option('display.max_colwidth', None)
combined_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>